In [1]:
import pandas as pd 

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
import numpy as np 

import re
import nltk
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import sent_tokenize,word_tokenize



In [2]:
df=pd.read_csv('./../Datasets/newData.csv')

df.head()

,Unnamed: 0,text,genre1,genre2,genre3
0,0,An exploratory dive into the deepest depths of...,Action,Science Fiction,Horror
1,1,Father Gabriele Amorth Chief Exorcist of the V...,Horror,Mystery,Thriller
2,2,When a new threat capable of destroying the en...,Action,Adventure,Science Fiction
3,3,Super-Hero partners Scott Lang and Hope van Dy...,Action,Adventure,Science Fiction
4,4,After dominating the boxing world Adonis Creed...,Drama,Action,NaN


In [3]:
def remove_tags(text):
    remove=re.compile(r'')
    return re.sub(remove, '', text)

df['text'] = df['text'].apply(remove_tags)

In [4]:
label_encoder = LabelEncoder()
df['genre1'] = label_encoder.fit_transform(df['genre1'])

In [5]:
def remove_splchr(text):
    cp=[]
    text=text.split()
    
    for i in text:
        if i.isalnum():
            cp.append(i)
        
    return " ".join(cp)

df['text']=df['text'].apply(remove_splchr)
    

In [7]:
df['text']=df['text'].str.lower()

In [8]:
input_data=df['text'].tolist()

In [9]:
input_data

['an exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a battle for on novel or entertainment warner pictures di bonaventura pictures cmc pictures gravity pictures china film group corporation',
 'father gabriele amorth chief exorcist of the vatican investigates a young terrifying possession and ends up uncovering a conspiracy the vatican has desperately tried to keep horrorscreen gems entertainment jesus mary worldwide katz loyola productions',
 'when a new threat capable of destroying the entire planet emerges optimus prime and the autobots must team up with a powerful faction known as the with the fate of humanity hanging in the balance humans noah and elena will do whatever it takes to help the transformers as they engage in the ultimate battle to save of the on on brother relationshipskydance paramount di bonaventura pictures bay films new republic pictu

In [10]:
corpus = []
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

for document in input_data:
    # Tokenization
    words = word_tokenize(document)

    # Remove stop words and perform stemming
    filtered_words = [stemmer.stem(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]

    # Join the processed words to form a document
    processed_document = " ".join(filtered_words)
    corpus.append(processed_document)

In [11]:
labels = df['genre1'].tolist()

In [12]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [13]:
padded_sequences = np.array(padded_sequences)
labels = np.array(labels)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [15]:
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [21]:
from tensorflow.keras.optimizers import Adam
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)

In [22]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=16, input_length=100))
model.add(LSTM(units=64, return_sequences=True))  # First LSTM layer with return_sequences=True
model.add(LSTM(units=32, return_sequences=True))  # Second LSTM layer with return_sequences=True
model.add(LSTM(units=16))  # Third LSTM layer
model.add(Dense(units=20, activation='softmax'))


model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
2825/2825 [==============================] - 189s 65ms/step - loss: 2.4298 - accuracy: 0.2415 - val_loss: 2.4149 - val_accuracy: 0.2444
Epoch 2/10
2825/2825 [==============================] - 182s 64ms/step - loss: 2.4236 - accuracy: 0.2415 - val_loss: 2.4127 - val_accuracy: 0.2444
Epoch 3/10
2825/2825 [==============================] - 176s 62ms/step - loss: 2.4231 - accuracy: 0.2415 - val_loss: 2.4132 - val_accuracy: 0.2444
Epoch 4/10
2825/2825 [==============================] - 187s 66ms/step - loss: 2.4232 - accuracy: 0.2415 - val_loss: 2.4132 - val_accuracy: 0.2444
Epoch 5/10
2825/2825 [==============================] - 176s 62ms/step - loss: 2.4228 - accuracy: 0.2415 - val_loss: 2.4130 - val_accuracy: 0.2444
Epoch 6/10
2825/2825 [==============================] - 170s 60ms/step - loss: 2.4230 - accuracy: 0.2415 - val_loss: 2.4130 - val_accuracy: 0.2444
Epoch 7/10
2825/2825 [==============================] - 162s 57ms/step - loss: 2.4227 - accuracy: 0.2415 - val_loss: 2